<p style="font-family: Arial; font-size:3em;color:black;"> Project</p>

We will use the titanic dataset from Kaggle (https://www.kaggle.com/). This is a well-known dataset and we will use it for classification- if the passenger survived or passed away.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
training_DF = pd.read_csv('titanic_dataset_GBC.csv')
training_DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
training_DF['Age'].fillna(training_DF['Age'].mean(), inplace=True)

In [39]:
training_DF.drop('Cabin',axis=1,inplace=True)

In [40]:
training_DF.dropna(inplace=True)

In [41]:
sex = pd.get_dummies(training_DF['Sex'])
embark = pd.get_dummies(training_DF['Embarked'])

In [42]:
training_DF.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [43]:
training_DF = pd.concat([training_DF,sex,embark],axis=1)

In [44]:
training_DF.drop(['female','C'],axis=1,inplace=True)

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(training_DF.drop(['Survived', 'Age','SibSp','Parch','male','Pclass',],axis=1), 
                                                    training_DF['Survived'], test_size=0.30, 
                                                    random_state=101)

In [47]:
# With the above steps, we have cleaned and pre-processed our data to bed fed to a ML/AI classification model.

# 1. Use Logistic Regression, Decision Tree, Random Forest, SGD, and SVM for classification of the dataset.
# 2. Use GridSearchCV to tune the parameter of each of the above model, can you obtain better results in this step for any of the models? Disucss you observations.
# 3. Randomly (or based on some hypothesis) remove some columns (1,2 or maybe 3 of them) of the dataset features and re-run your classification models. Do you observe any change in classification metrics?
# 4. Submit your project by the midnight of the 8th session. A suggested method is to submit a (text) report to discuss and summarize your findings and an .ipynb with clear comments and code sections to 
#    present your work.

# Importing necessary models and libraries

In [151]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Running all models with no parameters

In [156]:
models=[LogisticRegression(),DecisionTreeClassifier(),RandomForestClassifier(),SGDClassifier(),SVC()]
accuracy_scores=[]
for i in models:
    i.fit(X_train,y_train)
    accuracy_scores.append(accuracy_score(y_test,i.predict(X_test)))
accuracy_scores

[0.6779026217228464,
 0.5692883895131086,
 0.6404494382022472,
 0.6067415730337079,
 0.6367041198501873]

# Running GridSearchCv for Logistic Regression

In [287]:
def LR_Gridsearch(X_train,y_train):
    param_grid_lr={'C':np.linspace(0,100,100),'penalty': ['elasticnet','none'],'class_weight':['dict', 'balanced', 'None'],'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'l1_ratio':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
    gridlr=GridSearchCV(LogisticRegression(),param_grid_lr,refit=True,verbose=3,scoring='accuracy',n_jobs=-1)
    gridlr.fit(X_train,y_train)
    return gridlr.best_params_,gridlr.best_score_,gridlr.best_estimator_

In [288]:
lrparams,lrscore,lrmodel=LR_Gridsearch(X_train,y_train)

Fitting 5 folds for each of 33000 candidates, totalling 165000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1008 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 3568 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 7152 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 11760 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 17392 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 24048 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done 31728 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 40432 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 50160 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 60912 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 72688 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 85488 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 99312 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 11416

In [289]:
lrparams,lrscore

({'C': 0.0,
  'class_weight': 'dict',
  'l1_ratio': 0,
  'penalty': 'none',
  'solver': 'lbfgs'},
 0.6864516129032256)

In [290]:
lrpred=lrmodel.predict(X_test)
lr_accuracy=accuracy_score(y_test,lrpred)
lr_accuracy

0.6779026217228464

In [291]:
#testing for overfitting
accuracy_score(y_train,lrmodel.predict(X_train))

0.6864951768488746

Gridsearch always chooses minimum C for logistic regression

# Running GridSearchCv for Decision Tree

In [316]:
def DT_Gridsearch(X_train,y_train):
    param_grid_DT={'criterion':['gini', 'entropy'],'splitter':['best','random'],'max_depth':[1,2,3,4,5,6,7,8,9,10,'None'],'class_weight': ['dict', 'balanced', 'None'],'max_features' : [1,2,3,4,5]}
    gridDT=GridSearchCV(DecisionTreeClassifier(),param_grid_DT,refit=True,verbose=3,scoring='accuracy',n_jobs=-1)
    gridDT.fit(X_train,y_train)
    return gridDT.best_params_,gridDT.best_score_,gridDT.best_estimator_

In [317]:
bestparamsDT,bestscoresDT,DTmodel=DT_Gridsearch(X_train,y_train)

Fitting 5 folds for each of 660 candidates, totalling 3300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 3300 out of 3300 | elapsed:    4.8s finished


In [241]:
bestparamsDT,bestscoresDT,DTmodel

({'class_weight': 'balanced',
  'criterion': 'gini',
  'max_depth': 9,
  'max_features': 1,
  'splitter': 'random'},
 0.6719612903225807,
 DecisionTreeClassifier(class_weight='balanced', max_depth=9, max_features=1,
                        splitter='random'))

In [242]:
DTpred=DTmodel.predict(X_test)
DT_accuracy=accuracy_score(y_test,DTpred)
DT_accuracy

0.6254681647940075

In [243]:
#testing for overfitting
accuracy_score(y_train,DTmodel.predict(X_train))

0.6929260450160771

# Running GridSearchCv for Random Forest

In [318]:
def RF_Gridsearch(X_train,y_train):
    param_grid_RF={'n_estimators':[10,50,100,150,200],'criterion':['gini', 'entropy'],'max_depth':[1,2,3,4,5,6,7,8,9,10,'None'],'class_weight': ['dict', 'balanced', 'None'],'max_features' : [1,2,3,4],'min_samples_split':[1,2,3,4,5],'min_samples_leaf':[1,2,3,4,5]}
    gridRF=GridSearchCV(RandomForestClassifier(),param_grid_RF,refit=True,verbose=3,scoring='accuracy',n_jobs=-1)
    gridRF.fit(X_train,y_train)
    return gridRF.best_params_,gridRF.best_score_,gridRF.best_estimator_

In [319]:
bestparamsRF,bestscoresRF,RFmodel=RF_Gridsearch(X_train,y_train)
bestparamsRF,bestscoresRF

Fitting 5 folds for each of 33000 candidates, totalling 165000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 12272 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 26608 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 45040 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 56560 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 56976 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 57456 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 58000 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 58608 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 59280 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 60016 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 60816 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 61680 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 626

({'class_weight': 'balanced',
  'criterion': 'entropy',
  'max_depth': 10,
  'max_features': 4,
  'min_samples_leaf': 4,
  'min_samples_split': 2,
  'n_estimators': 10},
 0.6929161290322581)

In [320]:
RFpred=RFmodel.predict(X_test)
RF_accuracy=accuracy_score(y_test,RFpred)
RF_accuracy

0.6479400749063671

In [321]:
#testing for overfitting
accuracy_score(y_train,RFmodel.predict(X_train))

0.8344051446945338

# Running GridSearchCv for SGD

In [296]:
def SGD_Gridsearch(X_train,y_train):
    param_grid_SGD={'loss':['hinge','log','modified_huber','squared_hinge','perceptron','squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive'],'penalty': ['elasticnet'],'class_weight': ['dict', 'balanced', 'None'],'alpha':[0.0001,0.001,0.01,0.1,1,2,3,4,5],'l1_ratio':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
    gridSGD=GridSearchCV(SGDClassifier(),param_grid_SGD,refit=True,verbose=3,scoring='accuracy',n_jobs=-1)
    gridSGD.fit(X_train,y_train)
    return gridSGD.best_params_,gridSGD.best_score_,gridSGD.best_estimator_

In [297]:
bestparamsSGD,bestscoresSGD,SGDmodel = SGD_Gridsearch(X_train,y_train)
bestparamsSGD,bestscoresSGD

Fitting 5 folds for each of 2673 candidates, totalling 13365 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 6640 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 13188 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 13365 out of 13365 | elapsed:   13.5s finished


({'alpha': 2,
  'class_weight': 'balanced',
  'l1_ratio': 0.8,
  'loss': 'modified_huber',
  'penalty': 'elasticnet'},
 0.6719225806451613)

In [298]:
SGDpred=SGDmodel.predict(X_test)
SGD_accuracy=accuracy_score(y_test,SGDpred)
SGD_accuracy

0.6104868913857678

In [299]:
#testing for overfitting
accuracy_score(y_train,SGDmodel.predict(X_train))

0.6077170418006431

# Running GridSearchCv with Svc for kernels other than poly

In [257]:
def SVC_Gridsearch(X_train,y_train):
    param_grid_svc={'C':np.linspace(0.5,1,10),'kernel' : ['linear', 'rbf', 'sigmoid'],'gamma':[0.00001,0.0001,0.001,0.01,0.1,0,1]}
    grid=GridSearchCV(SVC(),param_grid_svc,refit=True,verbose=3,scoring='accuracy',n_jobs=-1)
    grid.fit(X_train,y_train)
    return grid.best_params_,grid.best_score_,grid.best_estimator_

In [258]:
bestparamssvc,bestscoresvc,svcmodel=SVC_Gridsearch(X_train,y_train)
bestparamssvc,bestscoresvc

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed: 16.4min finished


({'C': 0.7777777777777778, 'gamma': 0.0001, 'kernel': 'rbf'},
 0.6799999999999999)

In [259]:
svcpred=svcmodel.predict(X_test)
svc_accuracy=accuracy_score(y_test,svcpred)
svc_accuracy

0.6666666666666666

In [260]:
#testing for overfitting
accuracy_score(y_train,svcmodel.predict(X_train))

0.6945337620578779

# Comparing accuracies

In [322]:
Accuracys_data=pd.DataFrame({'no params':accuracy_scores,'GridSearchCV':[lr_accuracy,DT_accuracy,RF_accuracy,SGD_accuracy,svc_accuracy]},["Logistic Regression","Decision Tree","Random Forest","SGD","SVC"])
Accuracys_data

,no params,GridSearchCV
Logistic Regression,0.677903,0.677903
Decision Tree,0.569288,0.625468
Random Forest,0.640449,0.647940
SGD,0.606742,0.610487
SVC,0.636704,0.666667


# Dropping columns and re running GridsearchCV

In [323]:
#The features Q  and PassengerId were dropped because they were found to have little correlation with survival
X_train2=X_train.drop(['Q','PassengerId'],axis=1)
X_test2=X_test.drop(['Q','PassengerId'],axis=1)
paramsdropped=[]
scoresdropped=[]
modelsdropped=[]
accuracydropped=[]
for i in [0,1,2,3,4]:
    if i ==0:
        a,b,c=LR_Gridsearch(X_train2,y_train)
    if i ==1:
        a,b,c=DT_Gridsearch(X_train2,y_train)
    if i ==2:
        a,b,c=RF_Gridsearch(X_train2,y_train)
    if i ==3:
        a,b,c=SGD_Gridsearch(X_train2,y_train)
    if i ==4:
        a,b,c=SVC_Gridsearch(X_train2,y_train)
    paramsdropped.append(a)
    scoresdropped.append(b)
    modelsdropped.append(c)
    accuracydropped.append(accuracy_score(y_test,c.predict(X_test2)))

Fitting 5 folds for each of 33000 candidates, totalling 165000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 6640 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 13808 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 23024 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 34288 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 47600 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 62960 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 80368 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 99824 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 121328 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 144880 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 165000 out of 165000 | elapsed:  3.6min finished
C:\Users\Alber\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1315

Fitting 5 folds for each of 660 candidates, totalling 3300 fits


[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 3300 out of 3300 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s


Fitting 5 folds for each of 33000 candidates, totalling 165000 fits


[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 12272 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 26608 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 45040 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 56430 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 57448 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 58408 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 59496 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 60712 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 62056 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 63528 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 65128 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 66856 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 68712 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 70696 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 72808 tasks  

Fitting 5 folds for each of 2673 candidates, totalling 13365 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 6640 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 13219 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 13365 out of 13365 | elapsed:   13.8s finished


Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1008 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    9.0s finished


In [324]:
pd.DataFrame({'no params':accuracy_scores,'GridSearchCV':[lr_accuracy,DT_accuracy,RF_accuracy,SGD_accuracy,svc_accuracy],'Dropped Columns': accuracydropped},["Logistic Regression","Decision Tree","Random Forest","SGD","SVC"])

,no params,GridSearchCV,Dropped Columns
Logistic Regression,0.677903,0.677903,0.677903
Decision Tree,0.569288,0.625468,0.625468
Random Forest,0.640449,0.647940,0.629213
SGD,0.606742,0.610487,0.659176
SVC,0.636704,0.666667,0.674157


In [ ]:
#most models improved with dropped columns